In [1]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval
import random
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
%run MODEL_LIST.ipynb            #Load the models

In [3]:
df = pd.read_csv("../data/ufc-master.csv")

In [4]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [5]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [6]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [7]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[:-200]
odds_train = odds_df[:-200]
label_train = label_df[:-200]

df_test = df[-200:]
odds_test = odds_df[-200:]
label_test = label_df[-200:]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
3979
3979
3979


In [8]:
def get_ev(input_df, input_model, input_features, input_labels, odds_input, min_ev = 0, verbose=False, get_total=False):
    df_sel = input_df[input_features]
    df_sel = df_sel.dropna()
    df_sel = pd.get_dummies(df_sel)
    labels_sel = input_labels[input_labels.index.isin(df_sel.index)]
    odds_sel = odds_input[odds_input.index.isin(df_sel.index)] 
    best_score = custom_cv_eval(df_sel, input_model, labels_sel, odds_sel, min_ev = min_ev, verbose=verbose, 
                                get_total=get_total)
    return best_score

In [9]:
def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    best_ev = 0
    best_pos = -1
    for temp_ev in range(200):
        pos_ev = get_ev(input_df, input_model, input_features, input_labels, odds_input, min_ev=temp_ev, verbose=verbose,
                       get_total=True)
        print(temp_ev, pos_ev)
        if pos_ev > best_ev:
            best_ev = pos_ev
            best_pos = temp_ev
    return best_pos
    

In [13]:
ev = tune_ev(model_5, features_5, df_train, label_train, odds_train, verbose=False)

# Matches: 796, # Bets: 729 # Wins: 446
# Matches: 796, # Bets: 785 # Wins: 454
# Matches: 796, # Bets: 787 # Wins: 471
# Matches: 796, # Bets: 790 # Wins: 465
# Matches: 795, # Bets: 785 # Wins: 482
0 20082.41553788525
# Matches: 796, # Bets: 701 # Wins: 428
# Matches: 796, # Bets: 779 # Wins: 448
# Matches: 796, # Bets: 786 # Wins: 470
# Matches: 796, # Bets: 787 # Wins: 464
# Matches: 795, # Bets: 784 # Wins: 481
1 18872.889193509305
# Matches: 796, # Bets: 670 # Wins: 402
# Matches: 796, # Bets: 777 # Wins: 446
# Matches: 796, # Bets: 779 # Wins: 465
# Matches: 796, # Bets: 785 # Wins: 462
# Matches: 795, # Bets: 781 # Wins: 478
2 16701.40737185678
# Matches: 796, # Bets: 641 # Wins: 384
# Matches: 796, # Bets: 776 # Wins: 445
# Matches: 796, # Bets: 773 # Wins: 461
# Matches: 796, # Bets: 781 # Wins: 459
# Matches: 795, # Bets: 775 # Wins: 476
3 16424.538860362612
# Matches: 796, # Bets: 620 # Wins: 369
# Matches: 796, # Bets: 773 # Wins: 443
# Matches: 796, # Bets: 770 # Wins: 45

# Matches: 796, # Bets: 413 # Wins: 196
# Matches: 795, # Bets: 343 # Wins: 176
37 11011.39678064011
# Matches: 796, # Bets: 126 # Wins: 63
# Matches: 796, # Bets: 373 # Wins: 161
# Matches: 796, # Bets: 316 # Wins: 144
# Matches: 796, # Bets: 402 # Wins: 190
# Matches: 795, # Bets: 334 # Wins: 169
38 9734.086992977438
# Matches: 796, # Bets: 117 # Wins: 57
# Matches: 796, # Bets: 361 # Wins: 156
# Matches: 796, # Bets: 304 # Wins: 138
# Matches: 796, # Bets: 387 # Wins: 179
# Matches: 795, # Bets: 319 # Wins: 158
39 9116.894020734384
# Matches: 796, # Bets: 114 # Wins: 56
# Matches: 796, # Bets: 352 # Wins: 151
# Matches: 796, # Bets: 285 # Wins: 125
# Matches: 796, # Bets: 380 # Wins: 174
# Matches: 795, # Bets: 311 # Wins: 154
40 8722.51612251839
# Matches: 796, # Bets: 111 # Wins: 56
# Matches: 796, # Bets: 344 # Wins: 147
# Matches: 796, # Bets: 268 # Wins: 114
# Matches: 796, # Bets: 360 # Wins: 163
# Matches: 795, # Bets: 300 # Wins: 150
41 9056.452201169028
# Matches: 796, # Be

# Matches: 796, # Bets: 110 # Wins: 39
# Matches: 796, # Bets: 147 # Wins: 50
# Matches: 795, # Bets: 121 # Wins: 47
75 3043.731413514022
# Matches: 796, # Bets: 42 # Wins: 19
# Matches: 796, # Bets: 132 # Wins: 39
# Matches: 796, # Bets: 108 # Wins: 39
# Matches: 796, # Bets: 145 # Wins: 49
# Matches: 795, # Bets: 115 # Wins: 44
76 3068.493318275927
# Matches: 796, # Bets: 41 # Wins: 19
# Matches: 796, # Bets: 131 # Wins: 38
# Matches: 796, # Bets: 104 # Wins: 36
# Matches: 796, # Bets: 143 # Wins: 48
# Matches: 795, # Bets: 115 # Wins: 44
77 2663.4933182759264
# Matches: 796, # Bets: 40 # Wins: 19
# Matches: 796, # Bets: 130 # Wins: 37
# Matches: 796, # Bets: 104 # Wins: 36
# Matches: 796, # Bets: 139 # Wins: 45
# Matches: 795, # Bets: 110 # Wins: 41
78 2093.255223037832
# Matches: 796, # Bets: 38 # Wins: 19
# Matches: 796, # Bets: 129 # Wins: 37
# Matches: 796, # Bets: 101 # Wins: 36
# Matches: 796, # Bets: 137 # Wins: 43
# Matches: 795, # Bets: 107 # Wins: 40
79 2398.255223037832
#

# Matches: 796, # Bets: 15 # Wins: 5
# Matches: 796, # Bets: 74 # Wins: 25
# Matches: 796, # Bets: 51 # Wins: 19
# Matches: 796, # Bets: 68 # Wins: 18
# Matches: 795, # Bets: 54 # Wins: 16
115 2864.0
# Matches: 796, # Bets: 15 # Wins: 5
# Matches: 796, # Bets: 73 # Wins: 24
# Matches: 796, # Bets: 50 # Wins: 18
# Matches: 796, # Bets: 67 # Wins: 18
# Matches: 795, # Bets: 53 # Wins: 16
116 2678.0
# Matches: 796, # Bets: 14 # Wins: 5
# Matches: 796, # Bets: 72 # Wins: 24
# Matches: 796, # Bets: 49 # Wins: 17
# Matches: 796, # Bets: 63 # Wins: 16
# Matches: 795, # Bets: 52 # Wins: 16
117 2638.0
# Matches: 796, # Bets: 14 # Wins: 5
# Matches: 796, # Bets: 72 # Wins: 24
# Matches: 796, # Bets: 49 # Wins: 17
# Matches: 796, # Bets: 62 # Wins: 16
# Matches: 795, # Bets: 51 # Wins: 15
118 2428.0
# Matches: 796, # Bets: 14 # Wins: 5
# Matches: 796, # Bets: 72 # Wins: 24
# Matches: 796, # Bets: 47 # Wins: 16
# Matches: 796, # Bets: 61 # Wins: 15
# Matches: 795, # Bets: 49 # Wins: 14
119 1948.0


# Matches: 796, # Bets: 6 # Wins: 2
# Matches: 796, # Bets: 31 # Wins: 8
# Matches: 796, # Bets: 26 # Wins: 5
# Matches: 796, # Bets: 27 # Wins: 8
# Matches: 795, # Bets: 27 # Wins: 7
157 258.0
# Matches: 796, # Bets: 6 # Wins: 2
# Matches: 796, # Bets: 28 # Wins: 7
# Matches: 796, # Bets: 26 # Wins: 5
# Matches: 796, # Bets: 26 # Wins: 8
# Matches: 795, # Bets: 26 # Wins: 6
158 23.0
# Matches: 796, # Bets: 6 # Wins: 2
# Matches: 796, # Bets: 28 # Wins: 7
# Matches: 796, # Bets: 25 # Wins: 5
# Matches: 796, # Bets: 26 # Wins: 8
# Matches: 795, # Bets: 26 # Wins: 6
159 123.0
# Matches: 796, # Bets: 5 # Wins: 2
# Matches: 796, # Bets: 27 # Wins: 6
# Matches: 796, # Bets: 24 # Wins: 5
# Matches: 796, # Bets: 26 # Wins: 8
# Matches: 795, # Bets: 25 # Wins: 5
160 -52.0
# Matches: 796, # Bets: 5 # Wins: 2
# Matches: 796, # Bets: 27 # Wins: 6
# Matches: 796, # Bets: 24 # Wins: 5
# Matches: 796, # Bets: 25 # Wins: 8
# Matches: 795, # Bets: 24 # Wins: 5
161 148.0
# Matches: 796, # Bets: 5 # Win

# Matches: 796, # Bets: 12 # Wins: 2
# Matches: 796, # Bets: 13 # Wins: 2
# Matches: 796, # Bets: 12 # Wins: 3
# Matches: 795, # Bets: 9 # Wins: 1
199 -905.0


In [14]:
print(ev)

0
